In [1]:
import librosa
import librosa.display
import tensorflow as tf
import kapre
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import numpy as np
import pathlib
import os
import glob
import random
import math 
import soundfile as sf
import librosa 
from plotnine import *
from sklearn.metrics import confusion_matrix, mean_squared_error
import seaborn as sn
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.utils import shuffle
from tqdm.notebook import trange, tqdm
import tensorflow.keras.backend as K

In [2]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [3]:
seed = 27
tf.random.set_seed(seed)
random.seed(seed)
gpus = tf.config.list_physical_devices('GPU')

## need this if training on GPU
## tensorflow, get your shit together

if len(gpus)>0:

    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession

    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)

In [4]:
data_path = '/tf/data/ADReSS-IS2020'
audio_path_cc = data_path + '/Full_wave_enhanced_audio/cc/'
audio_path_cd = data_path + '/Full_wave_enhanced_audio/cd/'


text_path_cc = data_path + '/transcription/cc/'
text_path_cd = data_path + '/transcription/cd/'

# Meta Data

In [5]:
meta_cc = pd.read_csv(data_path + '/cc_meta_data.txt', sep=";", header=0, 
                      names = ['ID', 'Age', 'Gender', 'MMSE'])
meta_cd = pd.read_csv(data_path + '/cd_meta_data.txt', sep=";", header=0, 
                      names = ['ID', 'Age', 'Gender', 'MMSE'])

meta = meta_cc.assign(Group = 'cc').append(meta_cd.assign(Group = 'cd')).reset_index()
meta.MMSE = pd.to_numeric(meta.MMSE.replace(' NA', 30))
meta.ID = meta.ID.str.strip()

In [6]:
meta.sample(5)

,index,ID,Age,Gender,MMSE,Group
49,49,S071,74,male,29,cc
28,28,S036,73,male,28,cc
10,10,S013,57,male,30,cc
31,31,S040,66,female,30,cc
91,37,S136,79,male,23,cd


# Baseline 

In [7]:
filenames = list(glob.glob('/tf/data/ADReSS-IS2020/Full_wave_enhanced_audio/*/*'))
random.shuffle(filenames)

In [8]:
def getMelSpectrogramSamples(filename, n_mels=128, fmin=20, fmax=8000):
    y, sr = librosa.load(filename)

    ps = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmin=fmin, fmax=fmax, power=2)
    S_dB = librosa.power_to_db(ps, ref=np.max)
    
    #15 seconds
    window_size = 640
    window_hop = 160

    # truncate at start and end to only have windows full data
    # alternative would be to zero-pad
    start_frame = window_size 
    end_frame = window_hop * math.floor(float(S_dB.shape[1]) / window_hop)

    samples = np.array([S_dB[:, frame_idx-window_size:frame_idx] for frame_idx in range(start_frame, end_frame, window_hop)])
    
    return samples

def getSpectrogramSamples(filename):
    y, sr = librosa.load(filename)
    S = np.abs(librosa.stft(y, n_fft=512))
    S_dB = librosa.amplitude_to_db(S, ref=np.max)
    
    #15 seconds
    window_size = 640
    window_hop = 20

    # truncate at start and end to only have windows full data
    # alternative would be to zero-pad
    start_frame = window_size 
    end_frame = window_hop * math.floor(float(S_dB.shape[1]) / window_hop)

    samples = np.array([S_dB[:, frame_idx-window_size:frame_idx] for frame_idx in range(start_frame, end_frame, window_hop)])
    
    return samples

In [9]:
allSamples = []
ids = np.array([])
mmse_scores = np.array([])
classes = np.array([])

for file in tqdm(filenames):
    id = file.split('/')[-1].split('.')[0]
    mmse = meta.loc[meta.ID==id,"MMSE"].values[0]
    group = meta.loc[meta.ID==id,"Group"].values[0]
    
    samples = getMelSpectrogramSamples(file)
    allSamples.append(samples)
    
    numSamples = samples.shape[0]
    
    ids = np.append(ids, np.repeat(int(id[1:]), numSamples))
    mmse_scores = np.append(mmse_scores, np.repeat(mmse, numSamples))
    classes = np.append(classes, np.repeat(group, numSamples))

In [10]:
X = np.expand_dims(np.vstack(allSamples), -1)
y = mmse_scores

In [11]:
group_kfold = GroupKFold(n_splits=5)
group_kfold.get_n_splits(X, y, groups=ids)

5

In [12]:
train_index, test_index = next(group_kfold.split(X, y, groups=ids))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
ids_train, ids_test = ids[train_index], ids[test_index]

X_train, y_train, ids_train = shuffle(X_train, y_train, ids_train)
X_test, y_test, ids_test = shuffle(X_test, y_test, ids_test)

In [13]:
train_segment_mapping = {id:i for i, id in enumerate(np.unique(ids_train))}
segments_train = np.array([train_segment_mapping[id] for id in ids_train])

test_segment_mapping = {id:i for i, id in enumerate(np.unique(ids_test))}
segments_test = np.array([test_segment_mapping[id] for id in ids_test])

In [14]:
class SquareRootScheduler:
    def __init__(self, lr=0.1):
        self.lr = lr

    def __call__(self, num_update):
        return self.lr * pow(num_update + 1.0, -0.5)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(SquareRootScheduler(lr=0.001))

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    patience=10, 
    restore_best_weights=True)

In [15]:
def CustomWeightedLoss(y_true, y_pred, segments):
    
    y_true = tf.cast(y_true,tf.float32)
    y_pred = tf.cast(y_pred,tf.float32)
    segments = tf.cast(segments,tf.int32)
        
    grouped_y_true = tf.math.unsorted_segment_mean(
        y_true, segments, 1000
    )

    grouped_y_pred = tf.math.unsorted_segment_mean(
        y_pred, segments, 1000
    )

    # this returns the sum of squared errors across all samples. 
    # to find the actual RMSE, use this: lambda x: sqrt(X/num_samples)
    return tf.reduce_sum(tf.math.square(grouped_y_pred - grouped_y_true), axis=-1)

In [ ]:
inp = layers.Input(X.shape[1:])
true = layers.Input((1))
segment = layers.Input((1))

conv1 = layers.Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='conv1')(inp)
pool1 = layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pool1')(conv1)
flatten = layers.Flatten()(pool1)

dense1 = layers.Dense(128, activation='relu')(flatten)
dropout1 = layers.Dropout(0.5)(dense1)
dense2 = layers.Dense(32, activation='relu')(dropout1)
dropout2 = layers.Dropout(0.5)(dense2)
out = layers.Dense(1)(dropout2)

    
model = tf.keras.Model([inp, true, segment], out)

model.add_loss(CustomWeightedLoss(true, out, segment) )

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0),
    loss=None
)

In [20]:
history = model.fit(
    [X_train, y_train, segments_train], 
    y_train, 
    epochs=1000, 
    batch_size=64,
    verbose=1,
    callbacks = [es_callback, lr_callback]
)

Epoch 1/1000
21/21 [==============================] - 1s 57ms/step - loss: 10561.3291
Epoch 2/1000
21/21 [==============================] - 1s 57ms/step - loss: 9873.2324
Epoch 3/1000
21/21 [==============================] - 1s 59ms/step - loss: 8638.4932
Epoch 4/1000
21/21 [==============================] - 1s 57ms/step - loss: 9159.5518
Epoch 5/1000
21/21 [==============================] - 1s 59ms/step - loss: 7813.8779
Epoch 6/1000
21/21 [==============================] - 1s 61ms/step - loss: 7389.1279
Epoch 7/1000
21/21 [==============================] - 1s 60ms/step - loss: 6653.0552
Epoch 8/1000
21/21 [==============================] - 1s 59ms/step - loss: 7096.7148
Epoch 9/1000
21/21 [==============================] - 1s 59ms/step - loss: 6313.3496
Epoch 10/1000
21/21 [==============================] - 1s 59ms/step - loss: 7023.3081
Epoch 11/1000
21/21 [==============================] - 1s 59ms/step - loss: 6497.7129
Epoch 12/1000
21/21 [==============================] - 1s 58ms

In [21]:
def evaluateRMSE(model, X, y, segments):
    return np.sqrt(model.evaluate([X,y,segments],y)/len(np.unique(segments)))

In [22]:
evaluateRMSE(model, X_train, y_train, segments_train)

41/41 [==============================] - 0s 11ms/step - loss: 3390.8154


6.24298726543696

In [23]:
evaluateRMSE(model, X_test, y_test, segments_test)

11/11 [==============================] - 0s 10ms/step - loss: 1967.9429


9.680473402172984